# Mapping 3
- Line of Interest (circle & straight line)
- Frequency spectrum mapping
- Linear interpolation

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from IPython.display import Video
from pya import *

import simulation as sim

%matplotlib widget

In [ ]:
s = startup()

## Sonification

### Scanner functions
Return a scanner (function) for the sonification functions to use later.

The scanners recieve a timestamp and a normalized phase to return the simulation value on that at that timestamp and phase value.

In [ ]:

def line_of_interest_scanner(start_coordinates={'x': 0, 'y': 64}, end_coordinates={'x': 128, 'y': 64}, color=(0.25, 1, 0.25, 1)):
    """Returns a function that scans the simulation along a line"""
    start_array = np.array([start_coordinates['y'], start_coordinates['x']])[np.newaxis, :]
    end_array = np.array([end_coordinates['y'], end_coordinates['x']])[np.newaxis, :]

    def line_of_interest(simulation: sim.Simulation, t, phase_0_to_1):
        phase_0_to_1 = phase_0_to_1[:, np.newaxis]
        coordinates = (1 - phase_0_to_1) * start_array + phase_0_to_1 * end_array

        indices = np.array([t, coordinates[:, 0], coordinates[:, 1]])
        return simulation.interpolated(indices)
    
    # Patches for visual display
    additional_patches = [patches.FancyArrow(x=start_coordinates['x'], y=start_coordinates['y'], dx=end_coordinates['x']-start_coordinates['x'], dy=end_coordinates['y']-start_coordinates['y'], width=0.125, head_length=3, head_width=2, color=color)]

    return line_of_interest, additional_patches


def circle_of_interest_scanner(center_coordinates={'x': 64, 'y': 64}, radius=32, color=(0.25, 1, 0.25, 1)):
    """Returns a function that scans the simulation along a circle"""
    center_x = center_coordinates["x"]
    center_y = center_coordinates["y"]

    def circle_of_interest(simulation: sim.Simulation, t, phase_0_to_1):
        angle = 2*np.pi * phase_0_to_1
        x = center_x + radius * np.cos(angle)
        y = center_y + radius * np.sin(angle)
        
        indices = np.array([t, y, x])
        return simulation.interpolated(indices)
    
    # Patches for visual display
    additional_patches = [patches.Circle((center_coordinates['x'], center_coordinates['y']), radius=radius, edgecolor=color, linewidth=1, fill=False)]
    
    return circle_of_interest, additional_patches

### Path of Interest - Audification
Scans the simulation along the above defined paths at the given frequency. The result is directly used as signal.

Analogous to slicing the simulation at this path and using the result as wavetable.

In [ ]:
def audification(simulation: sim.Simulation, frequency=100, sample_rate=44100, scanner_fn=line_of_interest_scanner()[0], complex_to_real_fn=sim.probability_density):
    num_samples = int(sample_rate * simulation.duration_seconds())
    t = np.linspace(0, simulation.duration_seconds() * simulation.fps, num_samples, endpoint=False)
    phase_0_to_1 = np.linspace(0, frequency * simulation.duration_seconds(), num_samples, endpoint=False) % 1

    simulation_values = scanner_fn(simulation, t, phase_0_to_1)
    signal = complex_to_real_fn(simulation_values)
    
    return Asig(signal, sr=sample_rate)

### Timbre mapping

In [ ]:
def timbre_mapping(simulation: sim.Simulation, num_harmonics=32, spacing='log', frequency=100, sample_rate=44100, scanner_fn=line_of_interest_scanner()[0], complex_to_real_fn=sim.probability_density):
    num_samples = int(sample_rate * simulation.duration_seconds())
    t = np.linspace(0, simulation.duration_seconds() * simulation.fps, num_samples, endpoint=False)

    partial_number = np.arange(num_harmonics) + 1

    if (spacing == 'lin'): 
        positions = np.linspace(0, 1, num_harmonics)
    if (spacing == 'log'):
        positions = np.log2(partial_number) / np.log2(num_harmonics)

    # make 2d arrays to have each timestep have one entry for each position
    t = np.repeat(t[:, np.newaxis], num_harmonics, axis=1)
    positions = np.repeat(positions[np.newaxis, :], num_samples, axis=0)

    # flatten as scanner takes 1d inputs and rebuild the shape again
    simulation_values = scanner_fn(simulation, t.flatten(), positions.flatten()).reshape(num_samples, num_harmonics)
    amplitudes = complex_to_real_fn(simulation_values) / partial_number[np.newaxis, :]

    frequencies = np.reshape(frequency * partial_number, (1, -1))
    phases = np.reshape(np.random.random((num_harmonics)), (1, -1)) # random phase offsets to impulse like waveshape 
    video_seconds = np.reshape(np.linspace(0, simulation.duration_seconds(), num_samples, endpoint=False), (-1, 1))

    signal = np.sum(amplitudes * np.cos(2*np.pi * (video_seconds * frequencies + phases)), axis=1)
    return Asig(signal, sr=sample_rate)

### Field Audification

In [ ]:
def field_audification(simulation: sim.Simulation, scanline='vertical', x_stride=1, y_stride=1, time_stride=50, complex_to_real_fn=sim.probability_density):
    height = simulation.frames.shape[1]
    width = simulation.frames.shape[2]
    frames = simulation.frames[::time_stride, ::y_stride, ::x_stride] # apply stride
    
    if scanline == 'horizontal': frames = frames
    elif scanline == 'vertical': frames = frames.transpose(0, 2, 1)
    else: print("User warning: Specified scanline is invalid. Defaulting to vertical.")
    
    signal = complex_to_real_fn(frames).flatten()
    sample_rate = int(np.round(simulation.fps/time_stride * height/y_stride * width/x_stride))
    return Asig(signal, sample_rate)

## Scenarios
Note: Running the simulation and rendering the video can take about 30 seconds on an average device. Some sonifications also take that long.

### Scenario 1: Harmonic Oscillator

In [ ]:
harmonic_oscillator = sim.Simulation(
    title = "Harmonic Oscillator - Parabolic Potential",
    fps = 200,
    speed = 0.004,
    initial_state = sim.gaussian(),
    potential = sim.parabolic(),
    barrier = None,
    video_gamma=0.6
)
harmonic_oscillator.simulate(seconds=10)

In [ ]:
audification_scanner_fn, audification_patches = circle_of_interest_scanner(center_coordinates=dict(x=64, y=64), radius=32)
asig = audification(
    simulation=harmonic_oscillator, 
    frequency=100, 
    scanner_fn=audification_scanner_fn, 
    complex_to_real_fn=sim.probability_density
)
Video(harmonic_oscillator.video_with_sonification(asig, sonification_title="audification", additional_patches=audification_patches))

In [ ]:
timbre_mapping_scanner_fn, timbre_mapping_patches = line_of_interest_scanner(start_coordinates=dict(x=24, y=64), end_coordinates=dict(x=128-24, y=64))
asig = timbre_mapping(
    simulation=harmonic_oscillator,
    num_harmonics=10,
    spacing='log', # 'lin'
    frequency=100,
    scanner_fn=timbre_mapping_scanner_fn,
    complex_to_real_fn=sim.probability_density
)
Video(harmonic_oscillator.video_with_sonification(asig, sonification_title="timbre_mapping", additional_patches=timbre_mapping_patches))

In [ ]:
sonification_fps=8
asig = field_audification(
    simulation=harmonic_oscillator,
    scanline='vertical',
    x_stride=4,
    y_stride=4,
    time_stride=int(harmonic_oscillator.fps / sonification_fps),
    complex_to_real_fn=sim.probability_density
)
Video(harmonic_oscillator.video_with_sonification(asig, sonification_title="field_audification"))

### Scenario 2: Tunnel Effect

In [ ]:
tunnel_effect = sim.Simulation(
    title = "Tunnel Effect - Barrier and Parabolic Potential",
    fps = 230,
    speed = 0.004,
    initial_state = sim.gaussian(),
    potential = sim.parabolic(),
    barrier = sim.Barrier(x=64, width=1, slits=[]),
    video_gamma=0.6
)
tunnel_effect.simulate(seconds=10)

In [ ]:
audification_scanner_fn, audification_patches = circle_of_interest_scanner(center_coordinates=dict(x=64, y=64), radius=32)
asig = audification(
    simulation=tunnel_effect, 
    frequency=100, 
    scanner_fn=audification_scanner_fn, 
    complex_to_real_fn=sim.probability_density
)
Video(tunnel_effect.video_with_sonification(asig, sonification_title="audification", additional_patches=audification_patches))

In [ ]:
timbre_mapping_scanner_fn, timbre_mapping_patches = line_of_interest_scanner(start_coordinates=dict(x=24, y=64), end_coordinates=dict(x=128-24, y=64))
asig = timbre_mapping(
    simulation=tunnel_effect,
    num_harmonics=10,
    spacing='log', # 'lin'
    frequency=100,
    scanner_fn=timbre_mapping_scanner_fn,
    complex_to_real_fn=sim.probability_density
)
Video(tunnel_effect.video_with_sonification(asig, sonification_title="timbre_mapping", additional_patches=timbre_mapping_patches))

In [ ]:
sonification_fps=8
asig = field_audification(
    simulation=tunnel_effect,
    scanline='vertical',
    x_stride=4,
    y_stride=4,
    time_stride=int(harmonic_oscillator.fps / sonification_fps),
    complex_to_real_fn=sim.probability_density
)
Video(tunnel_effect.video_with_sonification(asig, sonification_title="field_audification"))

### Scenario 3: Single Slit

In [ ]:
single_slit = sim.Simulation(
    title = "Slingle Slit and Parabolic Potential",
    fps = 240,
    speed = 0.002,
    initial_state = sim.gaussian(),
    potential = sim.parabolic(),
    barrier = sim.Barrier(x=64, width=1, slits=[[64-5, 64+5]]),
    video_gamma=0.3
)
single_slit.simulate(seconds=10)

In [ ]:
audification_scanner_fn, audification_patches = circle_of_interest_scanner(center_coordinates=dict(x=64, y=64), radius=32)
asig = audification(
    simulation=single_slit, 
    frequency=100, 
    scanner_fn=audification_scanner_fn, 
    complex_to_real_fn=sim.probability_density
)
Video(single_slit.video_with_sonification(asig, sonification_title="audification", additional_patches=audification_patches))

In [ ]:
timbre_mapping_scanner_fn, timbre_mapping_patches = line_of_interest_scanner(start_coordinates=dict(x=24, y=64), end_coordinates=dict(x=128-24, y=64))
asig = timbre_mapping(
    simulation=single_slit,
    num_harmonics=10,
    spacing='log', # 'lin'
    frequency=100,
    scanner_fn=timbre_mapping_scanner_fn,
    complex_to_real_fn=sim.probability_density
)
Video(single_slit.video_with_sonification(asig, sonification_title="timbre_mapping", additional_patches=timbre_mapping_patches))

In [ ]:
sonification_fps=8
asig = field_audification(
    simulation=single_slit,
    scanline='vertical',
    x_stride=4,
    y_stride=4,
    time_stride=int(harmonic_oscillator.fps / sonification_fps),
    complex_to_real_fn=sim.probability_density
)
Video(single_slit.video_with_sonification(asig, sonification_title="field_audification"))

### Scenarion 4: Double Slit

In [ ]:
double_slit = sim.Simulation(
    title = "Double Slit and Parabolic Potential",
    fps = 240,
    speed = 0.002,
    initial_state = sim.gaussian(),
    potential = sim.parabolic(),
    barrier = sim.Barrier(x=64, width=1, slits=[[64-7, 64-5], [64+5, 64+7]]),
    video_gamma=0.3
)
double_slit.simulate(seconds=10)

In [ ]:
Video(double_slit.render_video(render_barrier=False, complex_to_real_fn=lambda psi: sim.probability_density(np.fft.fft2(psi))))

In [ ]:
audification_scanner_fn, audification_patches = circle_of_interest_scanner(center_coordinates=dict(x=64, y=64), radius=32)
asig = audification(
    simulation=double_slit, 
    frequency=100, 
    scanner_fn=audification_scanner_fn, 
    complex_to_real_fn=sim.probability_density
)
Video(double_slit.video_with_sonification(asig, sonification_title="audification", additional_patches=audification_patches))

In [ ]:
timbre_mapping_scanner_fn, timbre_mapping_patches = line_of_interest_scanner(start_coordinates=dict(x=24, y=64), end_coordinates=dict(x=128-24, y=64))
asig = timbre_mapping(
    simulation=double_slit,
    num_harmonics=10,
    spacing='log', # 'lin'
    frequency=100,
    scanner_fn=timbre_mapping_scanner_fn,
    complex_to_real_fn=sim.probability_density
)
Video(double_slit.video_with_sonification(asig, sonification_title="timbre_mapping", additional_patches=timbre_mapping_patches))

In [ ]:
sonification_fps=8
asig = field_audification(
    simulation=double_slit,
    scanline='vertical',
    x_stride=4,
    y_stride=4,
    time_stride=int(harmonic_oscillator.fps / sonification_fps),
    complex_to_real_fn=sim.probability_density
)
Video(double_slit.video_with_sonification(asig, sonification_title="field_audification"))

In [ ]:
plt.close()
plt.figure()
frame = sim.parabolic(center=dict(x=28, y=64), factor=dict(x=8, y=8))
plt.pcolormesh(np.abs(np.square(frame)), cmap='inferno', norm=matplotlib.colors.PowerNorm(vmin=0, vmax=np.max(np.abs(np.square(frame))), gamma=0.5))
# plt.pcolormesh(potential, vmin=0, vmax=20000)
plt.colorbar()
plt.show()